<h1> D7041E Lab2</h1>
Sune Larsson, sunlar-3@student.ltu.se

Elliot Eriksson, lelrek-1@student.ltu.se

# Word2Vec 

### Performance of Word2Vec
***

The dimensions used for evaluating Word2Vec performance were **[10, 250, 1000]**. Each dimension was tested over **five simulations**, yielding the following average accuracy:

- **Dimension: 10**, Average Accuracy: 49.50%, Average time: 54.68s
- **Dimension: 250**, Average Accuracy: 69.75%, Average time: 109.73s
- **Dimension: 1000**, Average Accuracy: 70.25%, Average time: 289.80s
- (Threshold frequency = 0.00055 for all measurements)

#### Analysis
- **10 Dimensions**: The model's limited capacity at 10 dimensions restricts its ability to identify synonyms effectively. However, it still significantly outperforms random guessing (20-25% accuracy).

- **250 Dimensions**: This configuration achieved the highest accuracy. It likely provides enough capacity to capture the relationships between words without introducing excessive noise.

- **1000 Dimensions**: Although better than 10 dimensions, the accuracy drops slightly compared to 250 dimensions. This may be due to redundancies, increased noise, or overfitting.

#### Conclusion
The **optimal dimensionality** lies around **250**, offering the best trade-off between capacity and noise. Further exploration around this dimensionality could reveal even better performance.


# Random Indexing Word Embedding Analysis with TOEFL 

### Explanation of the Word Size Problem

#### Problem
The original code was designed to work correctly **only with a window size of 2** due to **hardcoded logic** for handling neighbors. This made the code inflexible and unsuitable for other window sizes.

#### Issues
1. **Hardcoded Neighbor Handling**:
   - The code explicitly checks for neighbors up to **2 positions away**, making it impossible to generalize to different window sizes.

2. **Edge Cases**:
   - The code fails to handle cases where there are **fewer neighbors** than the window size, potentially causing errors or incorrect updates.

#### Solution
To generalize the code for any window size:
1. **Iterate Over Neighbors**:
   - Modify the logic to **iterate over neighbors** dynamically, based on the specified window size, both to the **left and right** of the focus word.

2. **Handle Edge Cases**:
   - Implement checks to ensure the code correctly handles **bounds** and uses **available neighbors**, even if there are fewer than the specified window size.

By generalizing the neighbor handling logic, the code can now work for **any window size**, making it more flexible and ensuring correct updates based on the chosen context window.


In [ ]:
while line != "":
            if line != "\n":
                lines.append(line.split())
                words = lines[2]
                length = len(words)
                i = 0
                while i < length:
                    if not (word_space.get(words[i]) is None):
                        k = 1
                        word_space_vector = word_space[words[i]]
                        while (i - k >= 0) and (k <= window_size): #process left neighbors of the focus word
                            word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i - k]], -1))         
                            k += 1
                        # Handle different situations if there was not enough neighbors on the left in the current line    
                        if k <= window_size and (len(lines[1])>0): 
                            if len(lines[1]) < 2:
                                if k != 1: #if one word on the left was already added
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[1][0]], -1)) #update word embedding
                                else:
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[1][0]], -1)) #update word embedding
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[0][len(lines[0]) - 1]], -1)) #update word embedding
                            else:
                                if k != 1:
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[1][len(lines[1]) - 1]], -1)) #update word embedding
                                else:
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[1][len(lines[1]) - 1]], -1)) #update word embedding
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                                np.roll(dictionary[lines[1][len(lines[1]) - 2]], -1)) #update word embedding

                        k = 1
                        while (i + k < length) and (k <= window_size): #process right neighbors of the focus word
                            word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i + k]], 1)) #update word embedding
                            k += 1
                        if k <= window_size:
                            if len(lines[3]) < 2:
                                if k != 1:
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                else:
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[4][0]], 1)) #update word embedding
                            else:
                                if k != 1:
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                else:
                                    word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][0]], 1)) #update word embedding
                                    word_space[words[i]] = np.add(word_space[words[i]],
                                                            np.roll(dictionary[lines[3][1]], 1))

                    i += 1
                lines.pop(0)
            line = text_file.readline()

This code generalizes the handling of neighbors by iterating up to the specified window size and checking for the presence of neighbors in the previous and next lines. This should allow the code to work for any window size.



In [ ]:
if line != "\n":
    lines.append(line.split())
    words = lines[2]
    length = len(words)
    i = 0
    while i < length:
        if not (word_space.get(words[i]) is None):
            k = 1
            word_space_vector = word_space[words[i]]
            # Process left neighbors of the focus word
            while (i - k >= 0) and (k <= window_size):
                word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i - k]], -1))
                k += 1
            # Handle different situations if there were not enough neighbors on the left in the current line
            if k <= window_size:
                for j in range(k, window_size + 1):
                    if len(lines[1]) >= j:
                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[1][len(lines[1]) - j]], -1))
                    elif len(lines[0]) > 0:
                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[0][len(lines[0]) - 1]], -1))

            k = 1
            # Process right neighbors of the focus word
            while (i + k < length) and (k <= window_size):
                word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[words[i + k]], 1))
                k += 1
            if k <= window_size:
                for j in range(k, window_size + 1):
                    if len(lines[3]) >= j:
                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[3][j - 1]], 1))
                    elif len(lines[4]) > 0:
                        word_space[words[i]] = np.add(word_space[words[i]], np.roll(dictionary[lines[4][0]], 1))
        i += 1

### Performance of Random Indexing (RI)
***

The following are the results for **three different dimensionalities** with Random Indexing (RI). Each measurement includes **average accuracy** and **average time** with five simulation.

- **1000 Dimensions**: **64.50%** accuracy, **34.96s** average time
- **4000 Dimensions**: **61.75%** accuracy, **47.98s** average time
- **10000 Dimensions**: **63.75%** accuracy, **82.81s** average time


#### Conclusion
- **Accuracy vs. Dimensionality**:
  - Accuracy **decreases** from **64.50%** (1000 dimensions) to **61.75%** (4000 dimensions). This drop suggests that the increased dimensionality might introduce noise or lead to overfitting.
  - At **10000 dimensions**, accuracy **increases slightly** to **63.75%**, indicating that while more dimensions can improve representation, they may also reintroduce complexity and noise, affecting generalization.

#### Final Conclusion: Computational Demand Comparison

#### Accuracy Comparison
By comparing both methods, **Word2Vec** achieved the **highest accuracy**, approximately **4%** better than **Random Indexing (RI)**.

#### Best Performance Results
The **best performance** for each method was achieved with the following settings:
- **Word2Vec**: **250 Dimensions**, **Accuracy: 71.5%**, **Run Time: 109.73s** (5 runs)
- **Random Indexing (RI)**: **4000 Dimensions**, **Accuracy: 67.5%**, **Run Time: 47.98s** (5 runs)

#### Final Conclusion
- If the primary objective is to achieve the **highest accuracy**, and **computational power is not a constraint**, then **Word2Vec** is the better option.
- Conversely, if **computational efficiency** and **limited resources** are the key factors, **Random Indexing (RI)** is preferable, as it achieves reasonable accuracy with much lower computational demand.